In [1]:
## Import packages
import os
import mne
import numpy as np
import scipy as sp
import scipy.io as sio
import matplotlib.pyplot as plt

from scipy.signal import butter, sosfiltfilt, sosfreqz  
from scipy.io import loadmat
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, roc_curve, auc

import scripts.classification_utils as utils
import scripts.CSP as csp
import scripts.SACSP as sacsp

In [2]:
## Define directories
data_dir = '/home/inffzy/Desktop/cogs189/cogs189_final_project/data'
results_dir = '/home/inffzy/Desktop/cogs189/cogs189_final_project/results'

In [3]:
## Raw data directory names
bc4_2a_processed_name = 'bci_competition_4_2a_processed'
bc3_3a_processed_name = 'bci_competition_3_3a_processed'

## Create list of all data paths
bc4_2a_processed_data_paths = []
bc3_3a_processed_data_paths = []

## Add bc4_2a data paths
for subject_idx in range(1, 10):
    subject_name = 'A0' + str(subject_idx) + 'T'
    bc4_2a_processed_data_paths.append(
        os.path.join(data_dir, bc4_2a_processed_name, subject_name + '.npz'))
    
## Add bc3_3a data paths
for subject_idx in range(1, 4):
    subject_name = 'bc3_3a_s' + str(subject_idx)
    bc3_3a_processed_data_paths.append(
        os.path.join(data_dir, bc3_3a_processed_name, subject_name + '.npz'))
    
all_data_paths = bc4_2a_processed_data_paths + bc3_3a_processed_data_paths

In [4]:
## Define constants
sampling_f = 250
label_left_hand = 769
label_right_hand = 770
num_samples = 10
test_split = 0.33
random_state = 42
num_cross_val = 5
num_experiment = 10

## Regular CSP Visualization

In [ ]:
data_npz = np.load(all_data_paths[0])
data = data_npz['processed_motor_imagery_data']
labels = data_npz['descriptions']

## Extract left and right hand epochs
data = data[np.any([labels == label_left_hand, labels == label_right_hand], axis=0)]
labels = labels[np.any([labels == label_left_hand, labels == label_right_hand], axis=0)]

print(data.shape)

data_ds = np.mean(data, axis=-1)
print(data_ds.shape)

data_ds_c1 = data_ds[labels == label_left_hand]
data_ds_c2 = data_ds[labels == label_right_hand]

plt.scatter(data_ds_c1[:, 0], data_ds_c1[:, 1])
plt.scatter(data_ds_c2[:, 0], data_ds_c2[:, 1])

In [ ]:
data_c1 = data[labels == label_left_hand]
data_c2 = data[labels == label_right_hand]
csp_transform = csp.CSP(data_c1, data_c2, n_top=1)
data_ = csp.apply_CSP_transform(data, csp_transform)

print(data_.shape)

data_ds = np.mean(data_, axis=-1)
print(data_ds.shape)

data_ds_c1 = data_ds[labels == label_left_hand]
data_ds_c2 = data_ds[labels == label_right_hand]

plt.scatter(data_ds_c1[:, 0], data_ds_c1[:, 1])
plt.scatter(data_ds_c2[:, 0], data_ds_c2[:, 1])

In [ ]:
extracted_features = csp.CSP_extract_features(data, csp_transform, n_top=1)

In [ ]:
extracted_features.shape

In [ ]:
data_c1 = data[labels == label_left_hand]
data_c2 = data[labels == label_right_hand]
csp_transform = csp.CSP2(data_c1, data_c2, n_top=1)
data_ = csp.apply_CSP_transform(data, csp_transform)

print(data_.shape)

data_ds = np.mean(data_, axis=-1)
print(data_ds.shape)

data_ds_c1 = data_ds[labels == label_left_hand]
data_ds_c2 = data_ds[labels == label_right_hand]

plt.scatter(data_ds_c1[:, 0], data_ds_c1[:, 1])
plt.scatter(data_ds_c2[:, 0], data_ds_c2[:, 1])

In [ ]:
data_c1 = data[labels == label_left_hand]
data_c2 = data[labels == label_right_hand]

c1_filter_pairs, c2_filter_pairs = sacsp.SACSP2(data_c1, data_c2)

In [ ]:
data_c1 = data[labels == label_left_hand]
data_c2 = data[labels == label_right_hand]

c1_spatial_filters, c2_spatial_filters = sacsp.SACSP2(data_c1, data_c2)

In [ ]:
c1_spatial_filters[0].shape

In [ ]:
plt.plot(c1_spatial_filters[0][:, 0])
plt.plot(c1_spatial_filters[0][:, 1])
plt.plot(c1_spatial_filters[0][:, 2])
plt.plot(c1_spatial_filters[1][:, 0])
plt.plot(c1_spatial_filters[1][:, 1])
plt.plot(c1_spatial_filters[1][:, 2])

In [ ]:
plt.plot(c1_spatial_filters[0][:, 1])

In [ ]:
c1_spatial_filters[0][:, 0]

In [ ]:
c1_spatial_filters[0][:, 1]

In [ ]:
len(c1_filter_pairs)

In [ ]:
c1_spectral_filter1 = c1_filter_pairs[0][0]
c1_spectral_filter2 = c1_filter_pairs[1][0]
c1_spectral_filter3 = c1_filter_pairs[2][0]

c2_spectral_filter1 = c2_filter_pairs[0][0]
c2_spatial_filter1 = c2_filter_pairs[0][1]

print(c1_spectral_filter1.shape)

In [ ]:
print(c1_spectral_filter1)

In [ ]:
freq = np.fft.fftfreq(500) * sampling_f

f1 = c1_spectral_filter1[1:100]
f2 = c1_spectral_filter2[1:100]
f3 = c1_spectral_filter3[1:100]

plt.plot(freq[1:100], f1 / np.linalg.norm(f1))
plt.plot(freq[1:100], f2 / np.linalg.norm(f2))
plt.plot(freq[1:100], f3 / np.linalg.norm(f3))

signal = c2_spectral_filter1[1:100]
plt.plot(freq[1:100], signal / np.linalg.norm(signal))

In [ ]:
c1_spatial_filter1 = c1_filter_pairs[0][1]
c1_spatial_filter2 = c1_filter_pairs[1][1]
c1_spatial_filter3 = c1_filter_pairs[2][1]

c2_spectral_filter1 = c2_filter_pairs[0][0]
c2_spatial_filter1 = c2_filter_pairs[0][1]

In [ ]:
plt.plot(c1_spatial_filter1)
plt.plot(c1_spatial_filter2)
plt.plot(c1_spatial_filter3)

In [ ]:
c1_spatial_filter1

In [ ]:
c1_spatial_filter2

### Note: each of the following classification experiment is run 10 times, each with a different random seed

## Binary Classification with LDA

In [ ]:
## results records training accuracy, cross validation average score, and testing accuracy for each subject
results = np.zeros((num_experiment, len(all_data_paths), 3))

for j in range(num_experiment):
    for i, data_path in enumerate(all_data_paths):
        
        ## Load data
        data_npz = np.load(data_path)
        data = data_npz['processed_motor_imagery_data']
        labels = data_npz['descriptions']
        
        ## Extract left and right hand epochs
        data = data[np.any([labels == label_left_hand, labels == label_right_hand], axis=0)]
        labels = labels[np.any([labels == label_left_hand, labels == label_right_hand], axis=0)]
    
        ## Downsample with windowed means
        data_ds = utils.windowed_means(data, num_samples)
        
        ## Flatten data
        data_ds_flattened = utils.flatten_dim12(data_ds)
        
        ## Prepare training and testing data
        X_train, X_test, y_train, y_test = train_test_split(data_ds_flattened, 
                                                            labels, 
                                                            test_size=test_split, 
                                                            shuffle=True,
                                                            random_state=j)
        
        ## Classification
        clf = utils.LDA_classifier(X_train, y_train, cross_val=num_cross_val)
        cross_val_score_avg = clf.train_binary()
        predictions, train_accuracy = clf.test_binary()
        predictions, test_accuracy = clf.test_binary(X_test, y_test)
        results[j, i, :] = [train_accuracy, cross_val_score_avg, test_accuracy]

In [ ]:
print(results)
results_avg = np.mean(results, axis=0)
print(results_avg)

results_path = os.path.join(results_dir, '2_LDA.npy')
np.save(results_path, results)

## Binary Classification with Regular CSP + LDA

In [ ]:
## results records training accuracy, cross validation average score, and testing accuracy for each subject
results = np.zeros((num_experiment, len(all_data_paths), 3))

for j in range(num_experiment):
    for i, data_path in enumerate(all_data_paths):
        
        ## Load data
        data_npz = np.load(data_path)
        data = data_npz['processed_motor_imagery_data']
        labels = data_npz['descriptions']
        
        ## Extract left and right hand epochs
        data = data[np.any([labels == label_left_hand, labels == label_right_hand], axis=0)]
        labels = labels[np.any([labels == label_left_hand, labels == label_right_hand], axis=0)]
        
        ## Prepare training and testing data
        X_train, X_test, y_train, y_test = train_test_split(data, 
                                                            labels, 
                                                            test_size=test_split, 
                                                            shuffle=True,
                                                            random_state=j)
        
        ## Classification
        clf = csp.CSP_LDA_classifier(X_train, y_train, cross_val=num_cross_val)
        cross_val_score_avg = clf.train_binary()
        predictions, train_accuracy = clf.test_binary()
        predictions, test_accuracy = clf.test_binary(X_test, y_test)
        results[j, i, :] = [train_accuracy, cross_val_score_avg, test_accuracy]

In [ ]:
print(results)
results_avg = np.mean(results, axis=0)
print(results_avg)

results_path = os.path.join(results_dir, '2_LDA_CSP.npy')
np.save(results_path, results)

## Binary Classification with Spectrally Adaptive CSP + LDA

In [7]:
## results records training accuracy, cross validation average score, and testing accuracy for each subject
results = np.zeros((num_experiment, len(all_data_paths), 3))

for j in range(num_experiment):
    for i, data_path in enumerate(all_data_paths):
        
        ## Load data
        data_npz = np.load(data_path)
        data = data_npz['processed_motor_imagery_data']
        labels = data_npz['descriptions']
        
        ## Extract left and right hand epochs
        data = data[np.any([labels == label_left_hand, labels == label_right_hand], axis=0)]
        labels = labels[np.any([labels == label_left_hand, labels == label_right_hand], axis=0)]
        
        ## Prepare training and testing data
        X_train, X_test, y_train, y_test = train_test_split(data, 
                                                            labels, 
                                                            test_size=test_split, 
                                                            shuffle=True,
                                                            random_state=j)
        
        ## Classification
        clf = sacsp.SACSP_LDA_classifier(X_train, y_train, cross_val=num_cross_val)
        cross_val_score_avg = clf.train_binary()
        predictions, train_accuracy = clf.test_binary()
        predictions, test_accuracy = clf.test_binary(X_test, y_test)
        results[j, i, :] = [train_accuracy, cross_val_score_avg, test_accuracy]

Cross validation score average:  0.7719298245614035
Training accuracy:  0.7934782608695652
Testing accuracy:  0.6739130434782609
Cross validation score average:  0.5824561403508772
Training accuracy:  0.6153846153846154
Testing accuracy:  0.37777777777777777
Cross validation score average:  0.9339181286549708
Training accuracy:  0.9230769230769231
Testing accuracy:  0.8695652173913043
Cross validation score average:  0.6052287581699346
Training accuracy:  0.6046511627906976
Testing accuracy:  0.5348837209302325
Cross validation score average:  0.6169934640522877
Training accuracy:  0.6162790697674418
Testing accuracy:  0.4186046511627907
Cross validation score average:  0.6666666666666666
Training accuracy:  0.7066666666666667
Testing accuracy:  0.4473684210526316
Cross validation score average:  0.5065359477124183
Training accuracy:  0.5056179775280899
Testing accuracy:  0.5454545454545454
Cross validation score average:  0.9202614379084967
Training accuracy:  0.9318181818181818
Testi

Testing accuracy:  0.5263157894736842
Cross validation score average:  0.6313725490196077
Training accuracy:  0.6404494382022472
Testing accuracy:  0.38636363636363635
Cross validation score average:  0.918954248366013
Training accuracy:  0.9318181818181818
Testing accuracy:  0.8636363636363636
Cross validation score average:  0.675
Training accuracy:  0.6753246753246753
Testing accuracy:  0.6666666666666666
Cross validation score average:  0.9577777777777777
Training accuracy:  0.9583333333333334
Testing accuracy:  0.88
Cross validation score average:  0.9
Training accuracy:  0.9354838709677419
Testing accuracy:  0.625
Cross validation score average:  1.0
Training accuracy:  1.0
Testing accuracy:  0.42857142857142855
Cross validation score average:  0.8169590643274856
Training accuracy:  0.7934782608695652
Testing accuracy:  0.6956521739130435
Cross validation score average:  0.5701754385964912
Training accuracy:  0.6263736263736264
Testing accuracy:  0.4222222222222222
Cross validati

In [8]:
print(results)
results_avg = np.mean(results, axis=0)
print(results_avg)

results_path = os.path.join(results_dir, '2_LDA_SACSP.npy')
np.save(results_path, results)

[[[0.79347826 0.77192982 0.67391304]
  [0.61538462 0.58245614 0.37777778]
  [0.92307692 0.93391813 0.86956522]
  [0.60465116 0.60522876 0.53488372]
  [0.61627907 0.61699346 0.41860465]
  [0.70666667 0.66666667 0.44736842]
  [0.50561798 0.50653595 0.54545455]
  [0.93181818 0.92026144 0.86363636]
  [0.76623377 0.76583333 0.74358974]
  [0.89583333 0.87555556 0.88      ]
  [0.87096774 0.87142857 0.625     ]
  [0.92592593 0.85333333 0.42857143]]

 [[0.79347826 0.72748538 0.67391304]
  [0.58241758 0.57309942 0.53333333]
  [0.87912088 0.83508772 0.91304348]
  [0.65116279 0.60718954 0.41860465]
  [0.65116279 0.61764706 0.44186047]
  [0.6        0.57333333 0.5       ]
  [0.61797753 0.62679739 0.56818182]
  [0.90909091 0.89869281 0.93181818]
  [0.74025974 0.72833333 0.71794872]
  [0.97916667 0.95555556 0.88      ]
  [0.87096774 0.87619048 0.5       ]
  [0.7037037  0.71333333 0.42857143]]

 [[0.73913043 0.73040936 0.69565217]
  [0.67032967 0.60409357 0.44444444]
  [0.89010989 0.87953216 0.8913043

## 4-Class One-vs-One Classification with LDA

In [ ]:
## results records training accuracy, cross validation average score, and testing accuracy for each subject
results = np.zeros((num_experiment, len(all_data_paths), 3))

for j in range(num_experiment):
    for i, data_path in enumerate(all_data_paths):
        
        ## Load data
        data_npz = np.load(data_path)
        data = data_npz['processed_motor_imagery_data']
        labels = data_npz['descriptions']
    
        ## Downsample with windowed means
        data_ds = utils.windowed_means(data, num_samples)
        
        ## Flatten data
        data_ds_flattened = utils.flatten_dim12(data_ds)
        
        ## Prepare training and testing data
        X_train, X_test, y_train, y_test = train_test_split(data_ds_flattened, 
                                                            labels, 
                                                            test_size=test_split, 
                                                            shuffle=True,
                                                            random_state=j)
        
        ## Classification
        clf = utils.LDA_classifier(X_train, y_train, cross_val=num_cross_val)
        cross_val_scores = clf.train_1_vs_1()
        predictions, train_accuracy = clf.test_1_vs_1()
        predictions, test_accuracy = clf.test_1_vs_1(X_test, y_test)
        results[j, i, :] = [train_accuracy, np.mean(cross_val_scores), test_accuracy]

In [ ]:
print(results)
results_avg = np.mean(results, axis=0)
print(results_avg)

results_path = os.path.join(results_dir, '4_LDA.npy')
np.save(results_path, results)

## 4-Class One-vs-One Classification with Regular CSP + LDA

In [ ]:
## results records training accuracy, cross validation average score, and testing accuracy for each subject
results = np.zeros((num_experiment, len(all_data_paths), 3))

for j in range(num_experiment):
    for i, data_path in enumerate(all_data_paths):
        
        ## Load data
        data_npz = np.load(data_path)
        data = data_npz['processed_motor_imagery_data']
        labels = data_npz['descriptions']
        
        ## Prepare training and testing data
        X_train, X_test, y_train, y_test = train_test_split(data, 
                                                            labels, 
                                                            test_size=test_split, 
                                                            shuffle=True,
                                                            random_state=j)
        
        ## Classification
        clf = csp.CSP_LDA_classifier(X_train, y_train, cross_val=num_cross_val)
        cross_val_scores = clf.train_1_vs_1()
        predictions, train_accuracy = clf.test_1_vs_1()
        predictions, test_accuracy = clf.test_1_vs_1(X_test, y_test)
        results[j, i, :] = [train_accuracy, np.mean(cross_val_scores), test_accuracy]

In [ ]:
print(results)
results_avg = np.mean(results, axis=0)
print(results_avg)

results_path = os.path.join(results_dir, '4_LDA_CSP.npy')
np.save(results_path, results)

## 4-Class One-vs-One Classification with Spectrally Adaptive CSP + LDA

In [9]:
## results records training accuracy, cross validation average score, and testing accuracy for each subject
results = np.zeros((num_experiment, len(all_data_paths), 3))

for j in range(num_experiment):
    for i, data_path in enumerate(all_data_paths):
        
        ## Load data
        data_npz = np.load(data_path)
        data = data_npz['processed_motor_imagery_data']
        labels = data_npz['descriptions']
        
        ## Prepare training and testing data
        X_train, X_test, y_train, y_test = train_test_split(data, 
                                                            labels, 
                                                            test_size=test_split, 
                                                            shuffle=True,
                                                            random_state=j)
        
        ## Classification
        clf = sacsp.SACSP_LDA_classifier(X_train, y_train, cross_val=num_cross_val)
        cross_val_scores = clf.train_1_vs_1()
        predictions, train_accuracy = clf.test_1_vs_1()
        predictions, test_accuracy = clf.test_1_vs_1(X_test, y_test)
        results[j, i, :] = [train_accuracy, np.mean(cross_val_scores), test_accuracy]

Cross validation scores for labels  769  and  770 :  0.7812865497076023
Cross validation scores for labels  769  and  771 :  0.802923976608187
Cross validation scores for labels  769  and  772 :  0.8467836257309941
Cross validation scores for labels  770  and  771 :  0.9894736842105264
Cross validation scores for labels  770  and  772 :  1.0
Cross validation scores for labels  771  and  772 :  0.7222222222222222
Training accuracy:  0.6758241758241759
Testing accuracy:  0.6153846153846154
Cross validation scores for labels  769  and  770 :  0.5884210526315788
Cross validation scores for labels  769  and  771 :  0.6385964912280702
Cross validation scores for labels  769  and  772 :  0.6432748538011696
Cross validation scores for labels  770  and  771 :  0.749019607843137
Cross validation scores for labels  770  and  772 :  0.681045751633987
Cross validation scores for labels  771  and  772 :  0.5558823529411765
Training accuracy:  0.42777777777777776
Testing accuracy:  0.3
Cross validati

Cross validation scores for labels  770  and  772 :  0.6345029239766082
Cross validation scores for labels  771  and  772 :  0.6772058823529412
Training accuracy:  0.44
Testing accuracy:  0.14942528735632185
Cross validation scores for labels  769  and  770 :  0.6174999999999999
Cross validation scores for labels  769  and  771 :  0.6849999999999998
Cross validation scores for labels  769  and  772 :  0.575
Cross validation scores for labels  770  and  771 :  0.6351648351648351
Cross validation scores for labels  770  and  772 :  0.6857142857142857
Cross validation scores for labels  771  and  772 :  0.7428571428571428
Training accuracy:  0.4931506849315068
Testing accuracy:  0.2328767123287671
Cross validation scores for labels  769  and  770 :  0.5888888888888889
Cross validation scores for labels  769  and  771 :  0.5102941176470589
Cross validation scores for labels  769  and  772 :  0.7967320261437909
Cross validation scores for labels  770  and  771 :  0.5964912280701755
Cross va

Cross validation scores for labels  769  and  771 :  0.96
Cross validation scores for labels  769  and  772 :  0.9155555555555555
Cross validation scores for labels  770  and  771 :  1.0
Cross validation scores for labels  770  and  772 :  1.0
Cross validation scores for labels  771  and  772 :  0.7509090909090907
Training accuracy:  0.8181818181818182
Testing accuracy:  0.76
Cross validation scores for labels  769  and  770 :  0.9666666666666668
Cross validation scores for labels  769  and  771 :  0.9333333333333333
Cross validation scores for labels  769  and  772 :  0.8466666666666667
Cross validation scores for labels  770  and  771 :  0.8571428571428571
Cross validation scores for labels  770  and  772 :  0.9
Cross validation scores for labels  771  and  772 :  0.9
Training accuracy:  0.8852459016393442
Testing accuracy:  0.7096774193548387
Cross validation scores for labels  769  and  770 :  0.82
Cross validation scores for labels  769  and  771 :  0.9666666666666668
Cross valida

Training accuracy:  0.46111111111111114
Testing accuracy:  0.26666666666666666
Cross validation scores for labels  769  and  770 :  0.79
Cross validation scores for labels  769  and  771 :  0.9134502923976608
Cross validation scores for labels  769  and  772 :  0.9140350877192981
Cross validation scores for labels  770  and  771 :  0.872549019607843
Cross validation scores for labels  770  and  772 :  0.8620915032679738
Cross validation scores for labels  771  and  772 :  0.7875
Training accuracy:  0.7833333333333333
Testing accuracy:  0.6777777777777778
Cross validation scores for labels  769  and  770 :  0.5967320261437908
Cross validation scores for labels  769  and  771 :  0.6625
Cross validation scores for labels  769  and  772 :  0.527485380116959
Cross validation scores for labels  770  and  771 :  0.6330882352941176
Cross validation scores for labels  770  and  772 :  0.6105263157894737
Cross validation scores for labels  771  and  772 :  0.5823529411764705
Training accuracy:  

Cross validation scores for labels  769  and  772 :  0.8479532163742689
Cross validation scores for labels  770  and  771 :  0.5915032679738561
Cross validation scores for labels  770  and  772 :  0.77
Cross validation scores for labels  771  and  772 :  0.8126315789473685
Training accuracy:  0.49171270718232046
Testing accuracy:  0.35555555555555557
Cross validation scores for labels  769  and  770 :  0.8933823529411764
Cross validation scores for labels  769  and  771 :  0.7915032679738563
Cross validation scores for labels  769  and  772 :  0.8627450980392156
Cross validation scores for labels  770  and  771 :  0.8875816993464053
Cross validation scores for labels  770  and  772 :  0.807843137254902
Cross validation scores for labels  771  and  772 :  0.8257309941520468
Training accuracy:  0.7102272727272727
Testing accuracy:  0.6022727272727273
Cross validation scores for labels  769  and  770 :  0.7705882352941177
Cross validation scores for labels  769  and  771 :  0.784166666666

Training accuracy:  0.8852459016393442
Testing accuracy:  0.5483870967741935
Cross validation scores for labels  769  and  770 :  0.8066666666666666
Cross validation scores for labels  769  and  771 :  0.96
Cross validation scores for labels  769  and  772 :  0.7333333333333333
Cross validation scores for labels  770  and  771 :  0.9266666666666667
Cross validation scores for labels  770  and  772 :  0.8761904761904763
Cross validation scores for labels  771  and  772 :  0.8333333333333334
Training accuracy:  0.8392857142857143
Testing accuracy:  0.25
Cross validation scores for labels  769  and  770 :  0.7093567251461989
Cross validation scores for labels  769  and  771 :  0.8189473684210526
Cross validation scores for labels  769  and  772 :  0.8777777777777779
Cross validation scores for labels  770  and  771 :  0.9573099415204679
Cross validation scores for labels  770  and  772 :  0.9882352941176471
Cross validation scores for labels  771  and  772 :  0.6973856209150326
Training a

Cross validation scores for labels  769  and  772 :  0.5457516339869282
Cross validation scores for labels  770  and  771 :  0.6274509803921569
Cross validation scores for labels  770  and  772 :  0.6467836257309941
Cross validation scores for labels  771  and  772 :  0.6713235294117647
Training accuracy:  0.4228571428571429
Testing accuracy:  0.2413793103448276
Cross validation scores for labels  769  and  770 :  0.5497076023391813
Cross validation scores for labels  769  and  771 :  0.6235294117647059
Cross validation scores for labels  769  and  772 :  0.6152046783625732
Cross validation scores for labels  770  and  771 :  0.5933823529411765
Cross validation scores for labels  770  and  772 :  0.6
Cross validation scores for labels  771  and  772 :  0.5227941176470589
Training accuracy:  0.4057142857142857
Testing accuracy:  0.2413793103448276
Cross validation scores for labels  769  and  770 :  0.6028571428571429
Cross validation scores for labels  769  and  771 :  0.75238095238095

Cross validation scores for labels  771  and  772 :  0.7647058823529411
Training accuracy:  0.7102272727272727
Testing accuracy:  0.6704545454545454
Cross validation scores for labels  769  and  770 :  0.6441666666666667
Cross validation scores for labels  769  and  771 :  0.72
Cross validation scores for labels  769  and  772 :  0.7725
Cross validation scores for labels  770  and  771 :  0.7083333333333333
Cross validation scores for labels  770  and  772 :  0.7823529411764706
Cross validation scores for labels  771  and  772 :  0.6941176470588235
Training accuracy:  0.5443037974683544
Testing accuracy:  0.4050632911392405
Cross validation scores for labels  769  and  770 :  0.9555555555555555
Cross validation scores for labels  769  and  771 :  0.9577777777777777
Cross validation scores for labels  769  and  772 :  0.9111111111111111
Cross validation scores for labels  770  and  771 :  0.9818181818181818
Cross validation scores for labels  770  and  772 :  1.0
Cross validation scores

In [10]:
print(results)
results_avg = np.mean(results, axis=0)
print(results_avg)

results_path = os.path.join(results_dir, '4_LDA_SACSP.npy')
np.save(results_path, results)

[[[0.67582418 0.85711501 0.61538462]
  [0.42777778 0.64270669 0.3       ]
  [0.72222222 0.85216718 0.65555556]
  [0.42857143 0.6413585  0.25287356]
  [0.40571429 0.63875917 0.24137931]
  [0.47945205 0.66770299 0.32876712]
  [0.45303867 0.66890838 0.31111111]
  [0.76136364 0.83839755 0.67045455]
  [0.58227848 0.76091503 0.40506329]
  [0.93939394 0.96171717 0.88      ]
  [0.85245902 0.89087302 0.48387097]
  [0.75       0.81714286 0.42857143]]

 [[0.67032967 0.86951611 0.62637363]
  [0.48333333 0.66190804 0.23333333]
  [0.75       0.87229102 0.68888889]
  [0.43428571 0.60894465 0.28735632]
  [0.44       0.64980292 0.14942529]
  [0.49315068 0.65687271 0.23287671]
  [0.45303867 0.6648065  0.37777778]
  [0.6875     0.81965013 0.68181818]
  [0.65189873 0.77298903 0.39240506]
  [0.91919192 0.95949495 0.86      ]
  [0.8852459  0.93222222 0.51612903]
  [0.85714286 0.87380952 0.42857143]]

 [[0.63186813 0.85806641 0.52747253]
  [0.5        0.68090242 0.24444444]
  [0.78333333 0.87851637 0.5666666